## Initiate the Toolbox
- the parameters set here will be used to collect command-specific data when the `Fetcher` executes the TET pattern.

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbol=["AMD", "AAPL"],
    interval="1d",
    start_date="2000-01-01",
    end_date="2021-02-20",
)
toolbox

In [ ]:
result = toolbox.technical.mandelbrot_channel(historical=False, window="1m")



In [ ]:
result

In [ ]:
result.to_polars()

In [ ]:
from humbldata.core.standard_models.abstract.humblobject import HumblObject


HumblObject()

In [ ]:
%load_ext line_profiler

%lprun -T lprof0 -f toolbox.technical.mandelbrot_channel toolbox.technical.mandelbrot_channel(historical=True, window="1m")
